먼저 알 것은 성경의 모든 예언은 사사로이 풀 것이 아니니 예언은 언제든지 사람의 뜻으로 낸 것이 아니요 오직 성령의 감동하심을 받은 사람들이 하나님께 받아 말한 것임이라 (벧후1:20-21)

-------

<center><img src="https://github.com/idebtor/KMOOC-ML/blob/master/ipynb/images/joyai/LectureNotes_ML.png?raw=true" width=1000></center>

__NOTE:__ The following materials have been compiled and adapted from the numerous sources including my own. Please help me to keep this tutorial up-to-date by reporting any issues or questions. Send any comments or criticisms to `idebtor@gmail.com` Your assistances and comments will be appreciated.


# Chapter 4.  손실함수와 경사하강법(Gradient Descent)

:본 단원은 참고문헌 (3)에서 발췌한 것입니다. 

--------------
경사 하강법에 사용한 손실 함수가 어떻게 사용된 것인지 알아봅니다. 경사 하강법은 정의된 손실 함수의 값이 최소가 되는 지점을 찾아가는 방법입니다. 여기서 손실 함수란 예측 값과 타깃 값의 차이를 함수로 정의한 것을 말합니다. 지금까지 우리가 사용한 방법인 '오차를 변화율에 곱하여 가중치와 편향 조정하기'는 제곱 오차라는 손실 함수를 미분한 것과 같습니다. 여기서는 '손실 함수'으로부터 경사 하강법을 접근하는 방법을 시도합니다. 

## 1. 손실 함수의 모든 것

제곱 오차(Squared Error)는 타깃(실제)값과 예측값을 뺀 다음 제곱한 것입니다. 제곱 오차를 수식으로 나타내면 다음과 같습니다. 

\begin{align}
SE = (y - \hat{y})^2  \tag{1}
\end{align}

__이때 제곱 오차가 최소가 되면 산점도 그래프를 가장 잘 표현한 직선이 그려집니다.__ 즉, 제곱 오차의 최솟값을 찾는 방법을 알면 모델을 쉽게 만들 수 있습니다. 다음은 제곱 오차와 직선의 관계를 나타낸 그래프입니다. 

<center><img src="https://github.com/idebtor/KMOOC-ML/blob/master/ipynb/images/joyai/ai4all-diabetes8.png?raw=true" width="400"></center>
<center>그림 1: 제곱 오차(SE)와 선형 회귀(직선)의 관계 </center>

__제곱 오차 함수의 최솟값을 알아내려면 기울기에 따라 함수의 값이 낮은 쪽으로 이동해야 합니다.__ 기울기를 구하려면 제곱 오차를 가중치나 편향에 대해 미분하면 됩니다.  다행히 제곱 오차는 가중치나 편향에 대해 미분이 가능합니다. 그러면 먼저 가중치에 대하여 제곱 오차를 미분해 보겠습니다. 


## 2. 가중치에 대하여 제곱 오차 미분하기

다음 식은 제곱 오차를 가중치($w$)에 대하여 편미분한 것입니다. 

\begin{align}
\frac{\partial{SE}}{\partial{w}}  
&= \frac{\partial}{\partial{w}} (y - \hat{y})^2   \qquad{\because{ f(g(x))' = f'(g(x))g'(x) } } \tag{2} \\  
&= 2(y - \hat{y}) \frac{\partial}{\partial{w}} (y - \hat{y}) \qquad{\because{\text{y is a constant}} } \\
&= 2(y - \hat{y}) (- \frac{\partial}{\partial{w}} \hat{y}) \\
&= 2(y - \hat{y}) (- \frac{\partial}{\partial{w}} (wx + b)) \qquad{\because{\text{x, b are constants}} } \\
&= 2(y - \hat{y}) (- x) \\
&= -2(y - \hat{y}) x \tag{3} \\
\end{align}


#### 편미분이란 미분의 대상 변수($w$)를 제외한 다른 변수($x, b$ 등)를 상수로 취급하여 미분한 것입니다. 이 때 $y$는 준비된 타깃 데이터이므로 $w$의 함수가 아니고 $\hat{y}$은 $w$의 함수($\hat{y} = wx + b$)입니다. 

따라서 $\hat{y}$을 $w$에 대해 미분하면 상수항 $b$는 사라지고 $x$만 남습니다. 결국 제곱 오차를 가중치($w$)에 대해 미분한 결과는 다음과 같습니다. 

\begin{align}
\frac{\partial{SE}}{\partial{w}} = -2(y - \hat{y}) x \tag{4}
\end{align}

최기에 제곱 오차 공식을 $(y - \hat{y})^2$가 아니라 $\frac{1}{2}(y - \hat{y})^2$로 정의했다면 2와 $\frac{1}{2}$이 서로 약분되어 조금 더 깔끔하게 $-(y-\hat{y})x$ 로 표현할 수 있었을 것입니다. 그래서, 보통은 제곱 오차 공식을 2로 나눈 함수를 편미분합니다. 여기에서도 아래와 같이 가중치에 대한 __제곱 오차의 변화율__은 $-(y-\hat{y})x$ 를 사용하도록 하겠습니다. 

\begin{align}
\frac{\partial{SE}}{\partial{w}} = -(y - \hat{y}) x \tag{5}
\end{align}

## 3. 미분한 오차 함수로 가중치 조정하기

가중치에 대한 제곱 오차의 변화율을 구했습니다. 이제 기존의 가중치를 조정하기 위하여, 앞에서 가중치를 조정할 때 변화율을 더했던 것과 마찬가지로 가중치를 조정합니다. 여기서는 __$w$에서 변화율을 뺍니다. $w$에서 변화율을 더하지 않고 빼는 이유는 손실 함수의 낮은 쪽으로 이동하고 싶기 때문입니다__. 


\begin{align}
w_{new} &= w - \frac{\partial{SE}}{\partial{w}}  \\
        &= w + (y - \hat{y}) x \tag{6}
\end{align}

위의 식을 어디서 본 것 같지 않나요? 

앞에서 오차 역전파를 알아보면 작성한 코드에 이와 같은 식이 있었습니다. 오차 역전파를 알아보며 적용했던 수식(`w + w_rate * error`) 혹은 (`w + x_i * error`) 은 사실 제곱 오차를 미분했던 것과 같았던 것입니다. 

아래 코드는 오차 역전파를 사용하여 가중치 w를 조정하는 코드입니다. 

```
y_hat = x_i * w + b  
error = y_i - y_hat  


w = w + x_i * error                 # w_rate = x_i  
```

실제 코딩에서, 반복문을 사용하여 여러 샘플에 적용해야 하므로, x 대신에 x_i, w_new 대신 w로 표기하였습니다.

## 4. 편향에 대하여 제곱 오차 미분하기

편향에 대하여 제곱 오차를 미분할 때는 처음부터 $\frac{1}{2}$를 곱한 제곱 오차 공식을 사용합니다. 유도식은 다음과 같습니다. 

\begin{align}
\frac{\partial{SE}}{\partial{b}}  
&= \frac{\partial}{\partial{b}} \frac{1}{2}(y - \hat{y})^2   \qquad{\because{ f(g(x))' = f'(g(x))g'(x) } } \tag{7} \\
&= (y - \hat{y}) \frac{\partial}{\partial{b}} (y - \hat{y}) \qquad{\because{\text{y is a constant}} } \\
&= (y - \hat{y}) (- \frac{\partial}{\partial{b}} \hat{y}) \\
&= (y - \hat{y}) (- \frac{\partial}{\partial{b}} (wx + b)) \qquad{\because{\text{w, x are constants}} } \\
&= (y - \hat{y}) (-1) \\
&= -(y - \hat{y}) 1 \tag{8} \\
\end{align}

미분 과정은 앞에 했던 $w$에 대한 편미분 과정과 매우 비슷합니다. 다만, 여기는 $w$, $x$가 상수가 됩니다. 가중치에서 변화율을 뺐던 이유와 같은 이유로 편향에서 변화율을 뺍니다. 

\begin{align}
b_{new} &= b - \frac{\partial{SE}}{\partial{b}}  \\
        &= b + (y - \hat{y})  \tag{9}
\end{align}

이 식도 앞에서 작성한 코드와 정확히 일치합니다. 다음 코드를 참고하십시오. 

```
error = y_i - y_hat
b = b + 1 * error
```

앞으로는 손실 함수에 대해 일일이 변화율의 값을 계산하는 대신 편미분을 사용하여 변화율을 계산합니다. 그리고 변화율은 인공지능 분야에서 특별히 Gradient(그레디언트, 경사)라고 부릅니다. 앞으로는 변화율이란 용어 대신 __Gradient__를 사용합니다. 

지금까지 경사 하강법을 사용한 회귀 문제를 변화율을 직접 구하는 방식과 미분을 사용한 방식(손실 함수)으로 검증해 보았습니다. 다음에는 지금까지 배운 내용을 모두 정리하여 하나의 파이썬 클래스를 만들어 봅니다.  

## Exercise 

### 1. 다른 방법으로 가중치($w$)에 대하여 제곱 오차 미분하기

위에서 제곱 오차($SE$)를 가중치($w$)에 대하여 편미분할 때, 미분 공식 $f(g(x))' = f'(g(x))g'(x)$을 처음에 사용하였습니다. 여기서 연습 문제에서는 $(y - \hat{y})^2$을 풀어서 나온 $(y - 2y\hat{y}  + \hat{y}^2)$ 식부터 식(3)구하는 편미분 과정을 아래에 반복하십시오. 마크 다운을 사용하여 구체적으로 미분 과정을 작성하십시오. 


\begin{align}
\frac{\partial{SE}}{\partial{w}}  
&= \frac{\partial}{\partial{w}} (y - \hat{y})^2    \\  
&= \frac{\partial}{\partial{w}}(y^2 - 2y\hat{y}  + \hat{y}^2) \\
&=  \\
&=  \\
&.... \\
&= -2(y - \hat{y}) x 
\end{align}

### 2. 다른 방법으로 편향($b$)에 대하여 제곱 오차 미분하기

위에서 제곱 오차($SE$)를 편향($b$)에 대하여 편미분할 때, 미분 공식 $f(g(x))' = f'(g(x))g'(x)$을 처음에 사용하였습니다. 여기서 연습 문제에서는 $(y - \hat{y})^2$을 풀어서 나온 $(y - 2y\hat{y}  + \hat{y}^2)$ 식부터 위의 식(8)구하는 편미분 과정을 아래에 반복하십시오. 마크 다운을 사용하여 구체적으로 미분 과정을 작성하십시오. 

\begin{align}
\frac{\partial{SE}}{\partial{b}}  
&= \frac{\partial}{\partial{b}} \frac{1}{2}(y - \hat{y})^2   \\
&= \frac{\partial}{\partial{b}}(y^2 - 2y\hat{y}  + \hat{y}^2) \\
&=  \\
&=  \\
&.... \\
&= -(y - \hat{y}) 1  \\
\end{align}

## 참고문헌

1. 케라스 창시자에게 배우는 딥러닝, 프랑소와 숄레, 길벗
1. 핸즈온 머신러닝, 오렐리앙 제롱, 한빛미디어
1. 딥러닝 입문, 박해선, 이지스 퍼블리싱
1. 파이썬으로 배우는 기계학습, 김영섭, K-MOOC

-------
__Be joyful always!__ 1 Thes.5:16